EE-311
======

Lab 5: Dimensionality Reduction
----------------------------------------

created by Zahra Farsijani and François Marelli on 25.03.2020

# Import libraries


In [ ]:
import sklearn
from sklearn.datasets import load_iris, load_wine, make_circles
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import sklearn.decomposition
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore', category=sklearn.exceptions.ConvergenceWarning)

# Dimensionality Reduction

Many Machine Learning problems involve thousands or even millions of features for
each training instance. Not only does this make training extremely slow, it can also
make it much harder to find a good solution, and to visualize the data. This problem is often
referred to as the curse of dimensionality. 

In this lab we will explore different types of methods to reduce the dimenionality of our data:

1. Filter Methods
2. Wrapper Methods
3. Feature extraction

There are two main approaches for feature selection:

- wrapper methods, in which the features are selected using knowledge of the model
- filter methods, in which the selection of features is independent of model used

## 1.1 Filter Methods

A simple approach for feature selection is the filter method, in which we the features of the dataset are ranked according to a scoring criterion.

Selecting only the highest scoring features reduces the dimensionality of the dataset.

We will test this method on the iris dataset with all features, for binary classification.

In [ ]:
data_X, data_y = load_iris(return_X_y=True)

mask = (data_y == 0) | (data_y == 1)

data_X = data_X[mask]
data_y = data_y[mask]

X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.15, random_state=0)

Since we have four features, we cannot effectively plot the data. 

Instead, we try to get more insights into the correlation between various features by plotting 2D graphs for each possible selection of 2 features in the dataset.

What do you learn about the data? Can you see which features matter for classification and which ones don't?

In [ ]:
df = sns.load_dataset("iris")
df = df[df.species != 'virginica']
sns.pairplot(df, hue="species")
plt.show()

### Pearson's correlation coefficient filter

A simple criterion to measure the information contained in the features is to compute the correlation between the features and the labels.

Compute this criterion for all the features in the dataset, then select the two most informative features and show the resulting dataset in a scatterplot.

According to the previous plot, which combination of features do you expect to be selected?

In [ ]:

# Your turn to code!




## 1.2 Wrapper Methods

Wrapper methods select the best features by iteratively training the model and adding or removing features on the go.

We will investigate the two basic versions of wrapper methods: Forward search and Backward search.

### Forward search

In this method, we start with having no feature in the model. In each iteration, we add the feature which best improves our model performance till the addition of a new feature does not bring any improvement anymore.

Write a code that implements forward feature selection on a given model and dataset.

In [ ]:
def forward_search(X_train, y_train, X_test, y_test, model):
    """
    Implement forward search feature selection.

    Parameters
    ----------    
    X_train, X_test : numpy array of shape [n_samples, n_features], datasets
        
    y_train, y_test : numpy array of shape [n_train_samples, ], labels (binary)
      
    model : a sklearn model implementing the .fit() and .score() functions (e.g. LogisticRegression())
    
    Returns
    -------
    numpy array of shape [n_features_selected,]
            Vector with the indexes of selected features, sorted by first selected
    """
    
    #######################################################
    # Your code here!
    
    
    
    return
    

Run your forward search algorithm on the iris dataset and print which features have been selected for a basic logistic regression model.

What dimensions are selected?

Did you expect it?

In [ ]:

# Code here!




#### The wine dataset

The iris dataset is too simple to demonstrate the use of wrapper methods.

We will use a bigger dataset called [the wine dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine).

How many features does this one have?

In [ ]:
# Load and prepare data
data_X_wine, data_y_wine = load_wine(return_X_y=True)

mask = (data_y_wine == 0) | (data_y_wine == 1)

data_X_wine = data_X_wine[mask]
data_y_wine = data_y_wine[mask]

X_train_wine, X_test_wine, y_train_wine, y_test_wine = train_test_split(data_X_wine, data_y_wine, test_size=0.35, random_state=3)

Use your forward search function to reduce the wine dataset using a linear regression model, and show what features have been selected.

Further reduce the dimensionality by keeping only the 2 most relevant features.

Then, compare the model accuracy for both selected features and ALL features for the wine dataset.

In [ ]:
model = LogisticRegression()

features = forward_search(X_train_wine, y_train_wine, X_test_wine, y_test_wine, model)


# Here you code!




### Backward Search (bonus)

In this method, as opposed to previous one, we start with using all features to train the model. 

In each iteration, we remove the feature which has the least effect on our model, untill removing of a new variable does not improve the performance of the model.

Write a code that implements forward feature selection on a given model and dataset. It can be very similar to your forward search!

In [ ]:
def backward_search(X_train, y_train, X_test, y_test, model):
    """
    Implement backward search feature selection.

    Parameters
    ----------    
    X_train, X_test : numpy array of shape [n_samples, n_features], datasets
        
    y_train, y_test : numpy array of shape [n_train_samples, ], labels (binary)
      
    model : a sklearn model implementing the .fit() and .score() functions (e.g. LogisticRegression())
    
    Returns
    -------
    numpy array of shape [n_features_selected,]
            Vector with the indexes of selected features, sorted by importance of feature
    """
    
    
    #######################################################
    # Code here
    
    
    
    return
    

Use your backward search function to reduce the wine dataset for a linear regression model.

Can you reduce it further down to 2 features only? (*Spoiler: yes you can, but how?*)

How does the ordering of features compare to the one obtained with the forward search?

In [ ]:
model = LogisticRegression()

features = backward_search(X_train_wine, y_train_wine, X_test_wine, y_test_wine, model)


# Here we code again!




## 2. Feature extraction using PCA

Where feature selection only tries to find the most important in the dataset, feature extraction actually combines multiple features to create new (better?) features.

We will focus on a well-known feature extraction method: the Principal Component Analysis (PCA).

PCA tries to define a new coordinate system such that the variance of the data is maximized along its main axes.

This then allows to select only the first few axes to reduce the dimension of the space while still keeping most of the representative information contained in the data.

**In this lab (and in the homework), we choose to not normalize 2D data for illustration purposes. We will later demonstrate the benefits of normalization on higher dimensionality datasets.**

For demonstration purposes, we will create a toy dataset that illustrates the concepts of PCA. 

Wait... There are no labels!? Is this important?

In [ ]:
data_X_toy = np.random.default_rng(0).multivariate_normal([1, 0.5], [[1, 0], [0, 25]], 500)

# Rotate the gaussian
theta = np.radians(60)
c, s = np.cos(theta), np.sin(theta)
R = np.array(((c, -s), (s, c)))
data_X_toy = data_X_toy.dot(R)
data_X_toy += (10, -5)

plt.scatter(data_X_toy[:,0], data_X_toy[:,1])
plt.grid()
plt.show()

PCA is computed by extracting the eigenvectors of the covariance matrix of the data. 

Write the code to compute the PCA and plot the principal directions on the graph using `draw_vector`. 

How do you interpret it? Can you tell which is the principal axis and how?

You can use:

`np.cov` to compute covariance

`np.linalg.eig` to perform eigen decomposition of a matrix

In [ ]:
def draw_vector(v0, v1, color='b', ax=None):
    """
    Draw a vector on a matplotlib graph.

    Parameters:
        v0: np.array of size (2), the starting point of the vector
        v1: np.array of size (2), the end point of the vector
        color: str, color code for the plotting
        ax: which axes to plot to, leave empty to use current figure
    """
    
    ax = ax or plt.gca()
    arrowprops=dict(ec=color,
                    arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops, color='r')


# Code here, you must.




A numerically more stable way to compute PCA is Singular Value Decomposition (SVD).

Compute the principal components of the data using `np.linalg.svd` and check that the result is identical.

Do not forget to center the data first (zero mean)! Why is it important? Try without that and see the result.

In [ ]:

# Program here




### Dimensionality reduction

We can use PCA to reduce the dimension of a dataset by projecting it onto its N first principal components.

Let us show it on the iris dataset. Use SVD to compute its principal components, and plot the percentage of information contained in each principal component (ratio of variance explained by each component).

How many features seem relevant?

You can use `plt.bar( . )` for the figure.

In [ ]:

# Your code here




We can choose to keep only the two first principal components to reduce the dimensionality of the dataset.

Reminder: to project the data into the reduced space, compute the matrix multiplication of the dataset matrix $X$ by the matrix $W_{d}$, defined as the matrix containing the first $d$ principal vectors (i.e., the matrix composed of the first $d$ right eigenvectors).

Reduce the iris dataset to a dimension of 2 using PCA and show the result as a scatterplot.

In [ ]:

# Time to shine




Fortunately enough, you do not have to write all this every time, as `sklearn.decomposition.PCA` is there for you!

Use it to reduce the iris dataset to a dimension of 2, and show a scatter of the result.

In [ ]:

# One more time


